In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import glob
import json
import pandas as pd
import geopandas as gpd
from scripts.tft_data import (open_cmip_tft, open_for_tft,
                              train_val_split_min_max)
from scripts.model_eval import (cmip_prediction, metric_df)


ws_file = gpd.read_file('../geo_data/great_db/geometry/russia_ws.gpkg')
ws_file = ws_file.set_index('gauge_id')

hydro_target = 'q_mm_day'

if hydro_target == 'lvl_mbs':
    static_parameters = ['for_pc_sse', 'crp_pc_sse',
                         'inu_pc_ult', 'ire_pc_sse',
                         'lka_pc_use', 'prm_pc_sse',
                         'pst_pc_sse', 'cly_pc_sav',
                         'slt_pc_sav', 'snd_pc_sav',
                         'kar_pc_sse', 'urb_pc_sse',
                         'gwt_cm_sav', 'lkv_mc_usu',
                         'rev_mc_usu', 'sgr_dk_sav',
                         'slp_dg_sav', 'ws_area',
                         'ele_mt_sav', 'height_bs']
    nc_variable = 'nc_all_h'
else:
    static_parameters = ['for_pc_sse', 'crp_pc_sse',
                         'inu_pc_ult', 'ire_pc_sse',
                         'lka_pc_use', 'prm_pc_sse',
                         'pst_pc_sse', 'cly_pc_sav',
                         'slt_pc_sav', 'snd_pc_sav',
                         'kar_pc_sse', 'urb_pc_sse',
                         'gwt_cm_sav', 'lkv_mc_usu',
                         'rev_mc_usu', 'sgr_dk_sav',
                         'slp_dg_sav', 'ws_area',
                         'ele_mt_sav']
    nc_variable = 'nc_all_q'

### Define cmip models

In [3]:
with open('./cmip_ranks.json') as json_file:
    cmip_models = json.load(json_file)
cmip_models = list(cmip_models.values())

meteo_inputs = dict()
for model in cmip_models:
    meteo_inputs[model] = [f'precipitation_{model}',
                           f'daily_maximum_near_surface_air_temperature_{model}',
                           f'daily_minimum_near_surface_air_temperature_{model}']

### Read data

In [4]:
# read era5 data
era_file = open_for_tft(
    nc_files=glob.glob(f'../geo_data/great_db/{nc_variable}/*.nc'),
    static_path='../geo_data/attributes/static_with_height.csv',
    area_index=ws_file.index,
    meteo_input=['prcp_e5l', 't_max_e5l', 't_min_e5l'],
    hydro_target=hydro_target,
    with_static=True,
    shuffle_static=False)
# read cmip files
model_file = open_cmip_tft(
    nc_files=glob.glob(f'../geo_data/nc_concat_cmip_2017-2030/*.nc'),
    static_path='../geo_data/attributes/static_with_height.csv',
    cmip_index=meteo_inputs['access_cm2'],
    era_index=['prcp_e5l', 't_max_e5l', 't_min_e5l'],
    area_index=ws_file.index)
# define neccessary datasets
train_val_pred = train_val_split_min_max(era_df=era_file,
                                         cmip_df=model_file,
                                         hydro_target=hydro_target,
                                         with_static=True)

### Run prediction

In [5]:
test_df = cmip_prediction(gauge_id='10620',
                          hydro_target=hydro_target,
                          static_parameters=static_parameters,
                          meteo_input=['prcp_e5l', 't_max_e5l', 't_min_e5l'],
                          model_checkpoint='./epoch=20-step=274596.ckpt',
                          train_dataset=train_val_pred['train_ds'],
                          pred_df=train_val_pred['pred_df'],
                          test_df=train_val_pred['test_df'],
                          scaler=train_val_pred['scaler'],
                          path_to_result='./DELETE ME')


/opt/conda/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator StandardScaler from version 1.2.2 when using version 1.1.3. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/utilities/migration/utils.py:49: PossibleUserWarning: The loaded checkpoint was produced with Lightning v2.0.2, which is newer than your current Lightning version: v1.9.5
  rank_zero_warn(
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/utilities/parsing.py:269: UserWarning: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
  rank_zero_warn(
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/utilities/parsing.py:269: UserW

In [6]:
gauge_id = '10620'
res_df = pd.read_csv(f'./DELETE ME/{gauge_id}.csv')
metric_df(gauge_id=gauge_id,
          predictions=res_df['pred'].dropna(),
          targets=res_df['obs'].dropna())

NSE       KGE         r     alpha      beta      RMSE        delta
10620 -8.527007 -0.136005  0.284411  1.232806  0.148977  2.373046 -1393.836746